# Building Classification's using Myria3d by Andrew Klearman 8/8/2024

This notebook uses the point classifications that were already generated by Myria3d and the classifications created by Terrascan to accurately detect buildings in a point cloud.

Follow along by running each cell sequentially. Note that some cells may take up to 30 seconds to run.

### Import Libraries

In [ ]:
#fundamental libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#LAS specific libraries
import open3d as o3d
import laspy

### Load Data

Before running this cell, add your desired file path where specified. Note that it is expected that this file is an output of Myria3d which contains the channels 'building' and 'confidence'

In [ ]:
dir = 'path/to/file.las' #This only an example Path, you should replace this with the path to your output LAS file
las = laspy.read(dir)
print('successfuly loaded file!')
print("number of points:", len(las))

### Visualize the point cloud:

This cell allows you to visulaize the file that we will be working with.

In [ ]:
xyzt = np.vstack((las.x, las.y, las.z))
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(xyzt.T)
pcd_center = pcd_o3d.get_center()
pcd_o3d.translate(-pcd_center)
o3d.visualization.draw_geometries([pcd_o3d])

Now, let's look at the points that Terrascan identified as buildings:
1. Terrascan somtimes confuses trees and buildings, look for erroneous points that resemble trees
2. Terrascan sometimes misses small structures, compare this visualization to this next one.

You can run the visualization cells as many times as you want

In [ ]:
mask = las['classification'] == 6
xyzt = np.vstack((las.x[mask], las.y[mask], las.z[mask]))
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(xyzt.T)
pcd_center = pcd_o3d.get_center()
pcd_o3d.translate(-pcd_center)
o3d.visualization.draw_geometries([pcd_o3d])

Now, we'll look at the predicts that use both Myria3d and Terrascan. Run the first cell to process the algorithm for detecting buildings. Then run the next cell to visualize the buildings detected by the new algorithm. Notice these things:

1. Myria3d is much better at detecting walls than Terrascan
2. This mixed strategy of Myria3d and Terrascan creates a very clean prediction, with less noise than either method.

In [ ]:
#algorithm for combining the predictions of Myria3d and terrascan.
def get_building_mask(las):
    both = np.logical_and(las['confidence'] == 6, las['classification'] == 6)
    others = np.logical_and(las['building'] >.1, las['classification'] == 6) #increasing the .1 in this line reduces noise created from Terrascan
    confirmed = both | others
    to_be_checked = ~confirmed & (las['building'] > .98) #increasing the .98 reduces noise created from Myria3d
    
    return confirmed, to_be_checked

confirmed, checked = get_building_mask(las)
print('ready to display predictions')

In [ ]:
mask = confirmed
xyzt = np.vstack((las.x[mask], las.y[mask], las.z[mask]))
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(xyzt.T)
pcd_center = pcd_o3d.get_center()
pcd_o3d.translate(-pcd_center)

o3d.visualization.draw_geometries([pcd_o3d])



### Viewing the probable points

One nice benefit of using Myria3d is that it also returns the probability of a point being a building. This allows us to generate a set of points that need to be checked manually as they should be marked as buildings, despite being classified differently by Terrascan. When you run this cell, the light blue points indicate points that need to be checked, while the dark blue points have already been confirmed as building. **Warning:** These points also contain a lot of noise, so chekcing them manually is important.

In [ ]:
mask = confirmed | checked
xyzt = np.vstack((las.x[mask], las.y[mask], las.z[mask]))
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(xyzt.T)
pcd_center = pcd_o3d.get_center()
pcd_o3d.translate(-pcd_center)

colors = plt.get_cmap("tab20")(checked[mask])
pcd_o3d.colors = o3d.utility.Vector3dVector(colors[:,:3])
o3d.visualization.draw_geometries([pcd_o3d])

After running all these cells, the points confirmed as buildings are stored in a variable called confirmed:

In [ ]:
print(confirmed)

And the points 